In [2]:
import torch

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)

Using device: mps


In [3]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

In [52]:
df = pd.read_csv("final_dataset_tech.csv.gz").drop("Unnamed: 0", axis=1)
df['time_stamp'] = pd.to_datetime(df['time_stamp'])
df["volatility"] = (df["volatility"] - df['volatility'].mean()) / df["volatility"].std()
df


,Stock,time_stamp,volatility,MA_1_log_return,MA_5_log_return,MA_22_log_return,MA_44_log_return,MA_1_volatility,MA_5_volatility,MA_22_volatility,MA_44_volatility,MA_1_volume,MA_5_volume,MA_22_volume,MA_44_volume
0,1,2020-03-06,1.298290,-0.011603,0.014552,0.000846,0.000877,0.001197,0.001936,0.001052,0.000802,4929.640368,5084.752690,4820.954299,4795.661361
1,1,2020-03-09,3.384507,0.028879,0.008139,0.001962,0.001219,0.001461,0.001623,0.001093,0.000825,4994.925196,5039.090881,4831.785040,4800.693345
2,1,2020-03-10,2.307772,0.008389,-0.000968,0.002626,0.001462,0.002529,0.001815,0.001180,0.000869,4924.218329,4990.899754,4841.835622,4804.024129
3,1,2020-03-11,1.768501,0.022914,0.012900,0.003370,0.001635,0.001978,0.001687,0.001248,0.000904,5101.090690,4985.515579,4862.155744,4812.535856
4,1,2020-03-12,5.477530,-0.014601,0.006796,0.003107,0.001434,0.001702,0.001773,0.001298,0.000933,5070.530383,5004.080993,4878.381499,4822.092545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664980,178,2025-09-24,-0.073667,-0.006592,-0.000556,0.005221,0.001720,0.000687,0.000814,0.000833,0.000873,3057.298140,2950.713931,2837.638705,2858.451023
664981,178,2025-09-25,0.018076,-0.004841,-0.004158,0.002922,0.001722,0.000760,0.000745,0.000823,0.000874,3106.709103,2985.308763,2848.354836,2871.871304
664982,178,2025-09-26,-0.180991,-0.024329,-0.010258,0.001936,0.001316,0.000807,0.000762,0.000830,0.000872,3226.019775,3090.855058,2873.852732,2882.683179
664983,178,2025-09-29,0.010319,0.009607,-0.006998,0.002705,0.000942,0.000705,0.000743,0.000834,0.000875,3100.716181,3121.269799,2897.020777,2891.673508


In [ ]:
#normalize day of year/365
#do not change the year
#Normalization z-scale normalization for each of the features,
#Normalization in volatiity would not give big boost
#with high volatility, look at residuals of volatility and put a time based
    #Actual - predicted = residuals (apply for both of them)

In [53]:
testing_years = [2020,2021,2022,2023,2024]
training_dataset = df[df["time_stamp"].dt.year.isin(testing_years)]
testing_dataset = df[df["time_stamp"].dt.year==2025]

In [66]:
y_train = training_dataset["volatility"].values
X_train_df = training_dataset.loc[:, training_dataset.columns != 'volatility']
X_train_df['time_stamp'] = X_train_df["time_stamp"].astype('int64') // 10**9
X_train_df[X_train_df.columns] = (X_train_df[X_train_df.columns] - X_train_df[X_train_df.columns].mean()) / X_train_df[X_train_df.columns].std()

/var/folders/vb/pdc3w9394cvftrnpc2zrn3dc0000gn/T/ipykernel_60075/2529267694.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_df['time_stamp'] = X_train_df["time_stamp"].astype('int64') // 10**9
/var/folders/vb/pdc3w9394cvftrnpc2zrn3dc0000gn/T/ipykernel_60075/2529267694.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_df[X_train_df.columns] = (X_train_df[X_train_df.columns] - X_train_df[X_train_df.columns].mean()) / X_train_df[X_train_df.columns].std()


In [67]:
y_test = testing_dataset["volatility"].values
X_test_df = testing_dataset.loc[:, testing_dataset.columns != 'volatility']
X_test_df['time_stamp'] = X_test_df["time_stamp"].astype('int64') // 10**9
X_test_df[X_test_df.columns] = (X_test_df[X_test_df.columns] - X_test_df[X_test_df.columns].mean()) / X_test_df[X_test_df.columns].std()
X_test_df

/var/folders/vb/pdc3w9394cvftrnpc2zrn3dc0000gn/T/ipykernel_60075/1241999720.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_df['time_stamp'] = X_test_df["time_stamp"].astype('int64') // 10**9
/var/folders/vb/pdc3w9394cvftrnpc2zrn3dc0000gn/T/ipykernel_60075/1241999720.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_df[X_test_df.columns] = (X_test_df[X_test_df.columns] - X_test_df[X_test_df.columns].mean()) / X_test_df[X_test_df.columns].std()


,Stock,time_stamp,MA_1_log_return,MA_5_log_return,MA_22_log_return,MA_44_log_return,MA_1_volatility,MA_5_volatility,MA_22_volatility,MA_44_volatility,MA_1_volume,MA_5_volume,MA_22_volume,MA_44_volume
1214,-1.720970,-1.743371,-0.388356,-3.298895,-0.473679,-0.021692,-0.702458,2.758504,-0.039946,-0.531304,1.627598,1.457961,1.694006,1.782305
1215,-1.720970,-1.730590,-1.281828,-0.787459,-0.849046,-0.195713,-0.225206,-0.708400,0.009310,-0.503653,1.991296,1.760766,1.756487,1.792181
1216,-1.720970,-1.692246,-0.207194,-0.956437,-0.973556,-0.141889,-0.529352,-0.622959,0.015906,-0.497836,1.775048,1.808723,1.759480,1.792399
1217,-1.720970,-1.679464,0.069239,-0.772709,-1.062332,-0.025898,-0.634913,-0.637026,0.033718,-0.507455,1.698163,1.798733,1.759278,1.788222
1218,-1.720970,-1.666683,-0.217879,-0.897464,-1.089894,-0.120422,-0.578093,-0.645530,0.054632,-0.531669,1.742184,1.818094,1.761337,1.782593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664980,-0.430199,1.643689,-0.365934,-0.080497,1.170552,0.518256,-0.217738,0.087085,0.167795,0.323323,0.131684,-0.002052,-0.145059,-0.109051
664981,-0.430199,1.656470,-0.270045,-0.516262,0.648352,0.518857,-0.052806,-0.102672,0.131811,0.328304,0.196304,0.044353,-0.130377,-0.090496
664982,-0.430199,1.669252,-1.337486,-1.254369,0.424380,0.393943,0.054165,-0.055959,0.159048,0.319442,0.352339,0.185932,-0.095445,-0.075547
664983,-0.430199,1.707596,0.521357,-0.859872,0.599022,0.278884,-0.177944,-0.109994,0.170224,0.330988,0.188467,0.226731,-0.063704,-0.063117


In [ ]:
df = pd.read_csv("training_dataset_tech.csv")
print(df.shape)
y_train = df['volatility'].values
X_train = df.loc[:, df.columns != 'volatility'].values

X_tensor_train = torch.tensor(X_train, dtype=torch.float32)
y_tensor_train = torch.tensor(y_train, dtype=torch.float)

(576646, 16)


In [23]:
y_train.std()

np.float64(0.0005217733928955276)

In [4]:
df = pd.read_csv("testing_dataset_tech.csv")
y_test = df['volatility'].values
X_test = df.loc[:, df.columns != 'volatility'].values

X_tensor_test = torch.tensor(X_test, dtype=torch.float32)
y_tensor_test = torch.tensor(y_test, dtype=torch.float)

In [5]:
#figure out train test split and fine tuning
training_data = TensorDataset(X_tensor_train, y_tensor_train)
testing_data = TensorDataset(X_tensor_test, y_tensor_test)

In [6]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(testing_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 15])
Shape of y: torch.Size([64]) torch.float32


In [ ]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(15, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.layers(x)

model = NeuralNetwork().to(device)
print(model)

Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layers): Sequential(
    (0): Linear(in_features=15, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [4]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x, h0=None, c0=None):
        if h0 is None or c0 is None:
            h0 = torch.zeros(self.layer_dim, x.size(
                0), self.hidden_dim).to(x.device)
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out, hn, cn

In [ ]:
model = LSTMModel(input_dim=)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [16]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 1000 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [20]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss = 0
    preds = []
    targets = []
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            preds.append(pred.cpu())
            targets.append(y.cpu())
    test_loss /= num_batches
    pred_np = torch.cat(preds).numpy()
    y_np = torch.cat(targets).numpy()

    r_squared = r2_score(y_np, pred_np)
    print(f"Test Error: \n R-Squared: {r_squared}%, Avg loss: {test_loss:>8f} \n")

In [17]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
loss: 0.003202  [   64/576646]
loss: 0.001526  [64064/576646]
loss: 0.000468  [128064/576646]
loss: 0.000068  [192064/576646]
loss: 0.000002  [256064/576646]
loss: 0.000000  [320064/576646]
loss: 0.000000  [384064/576646]
loss: 0.000000  [448064/576646]
loss: 0.000000  [512064/576646]
loss: 0.000000  [576064/576646]
Epoch 2
-------------------------------
loss: 0.000001  [   64/576646]
loss: 0.000000  [64064/576646]
loss: 0.000000  [128064/576646]
loss: 0.000000  [192064/576646]
loss: 0.000000  [256064/576646]
loss: 0.000000  [320064/576646]
loss: 0.000000  [384064/576646]
loss: 0.000000  [448064/576646]
loss: 0.000000  [512064/576646]
loss: 0.000000  [576064/576646]
Epoch 3
-------------------------------
loss: 0.000001  [   64/576646]
loss: 0.000000  [64064/576646]
loss: 0.000000  [128064/576646]
loss: 0.000000  [192064/576646]
loss: 0.000000  [256064/576646]
loss: 0.000000  [320064/576646]
loss: 0.000001  [384064/576646]
loss: 0.000000  [44806

In [21]:
test(test_dataloader, model, loss_fn)

Test Error: 
 R-Squared: -0.6514965295791626%, Avg loss: 0.000000 



In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>